# 実験を終了する

現在の実験パッケージをGIN-forkに保存して、当実験実行環境を停止・削除します。

「実験フロートップページ」に戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。

Jupyter Notebookの操作方法については、[こちら](https://support.rdm.nii.ac.jp/usermanual/60/#jp_operation)をご覧ください。新規タブで開きます。

## 0. 研究リポジトリ名・実験パッケージ名を確認する  
以下のセルを実行すると、当実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  


In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
utils.show_name('blue', EXPERIMENT_TITLE)

## 1. 実行環境削除の準備をする

### 1-1. 実行環境構成を記録する

Pythonの実行環境の構成をenvironment.ymlおよびrequirements.txtに記録します。

In [ ]:
%store -r EXPERIMENT_TITLE
# Export environment configuration information(conda base env)
!conda env export -n base > ~/experiments/$EXPERIMENT_TITLE/environment.yml
!pip freeze > ~/experiments/$EXPERIMENT_TITLE/requirements.txt

### 1-2. READMEから実行環境へのリンクを削除する

In [ ]:
import subprocess
import urllib
from IPython.display import clear_output
import os
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util

# READMEに遷移リンクがあれば削除してから、新しく実行環境のリンクを追記する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE
path = experiment_path + '/README.md'
s = ''
http_url = subprocess.getoutput('git config --get remote.origin.url')
with open(path, 'r') as f:
    s = f.read()
    s = s[:s.find('## 実験実行環境にアクセスしたい場合')]

with open(path, 'w') as f:
    f.write(s)

with open(path, 'a', newline='\n') as f:
    f.write("\n## 実験実行環境にアクセスしたい場合")
    f.write("\n実行環境は削除されています。再度実験を実施する場合は以下のリンクをクリックして実験実行環境を再構築してください。別の環境へ移るため、新しいタブまたはウィンドウで開くことを推奨します。<br>")
    f.write("\n再構築が完了すると、「実験実行環境を再構築した場合のセットアップを行う」の実行画面が出力されますので、ガイドに従って実行してください。<br>")
    f.write("\nhttps://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(http_url, safe='') + "/HEAD?filepath=WORKFLOWS/EX-WORKFLOWS/util/required_rebuild_container.ipynb")

clear_output()
display_util.display_info('READMEから実行環境へのリンクを削除しました。')

## 2. 実験パッケージをGIN-forkに同期する

実験パッケージをGIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE
os.chdir(experiment_path)

#**************************************************#
#* Generate a list of folder paths to be managed by Git-annex. #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# Recursively search under the experimental package to obtain a list of absolute directory paths.
for root, dirs, files in os.walk(top=experiment_path):
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )


# Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
output_data_path = [ s for s in dirlist if 'output_data' in s ]
for output_data in output_data_path:
    if  "input_data" not in output_data:
        annexed_save_path.append( output_data )

# Add the input_data directory to annexed_save_path.
annexed_save_path.append( experiment_path + '/input_data'  )

# Generate a list of file paths to which metadata is to be assigned.
gitannex_files = []
for path in annexed_save_path:
    gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
             if os.path.isfile(p)]

#********************************************************#
#* Generate a list of directory paths and file paths to be managed by Git. #
#********************************************************#
# Obtain a list of directories and files directly under the experimental package.
files = os.listdir()

# Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')

for dirname in dirs:
    if dirname != 'ci' and dirname != 'source':
        full_param_dir = '{}/{}/params'.format(experiment_path,dirname)
        if os.path.isdir(full_param_dir):
            dirs.remove(dirname)
            ex_param_path = '{}/{}'.format(experiment_path, dirname)
            ex_param_path_childs = os.listdir(ex_param_path)
            for ex_param_path_child in ex_param_path_childs:
                if ex_param_path_child != 'output_data':
                    dirs.append('{}/{}'.format(dirname,ex_param_path_child))

# Obtain files directly under the experimental package.
files = [f for f in files if os.path.isfile(f)]

# Generate a list of folder paths and file paths to be managed by Git.
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[GIN-forkへの同期の失敗を解消する](../conflict_helper.ipynb)を参照して、競合を解消してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

# save this notebook
save_path.append('WORKFLOWS/EX-WORKFLOWS/finish.ipynb')
is_ok = utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_実験終了',get_paths=[])

## 3. 実験実行環境を停止・削除する

### 3-1. 実験実行環境を確認する

以下のセルを実行して実行環境のサーバー名を確認して下さい。  

In [ ]:
import os
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util
display_util.display_html_msg(msg=os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3], tag='h2')

### 3-2. コントロールパネルへ遷移し、実験実行環境を停止・削除する

[コントロールパネル](https://jupyter.cs.rcos.nii.ac.jp/hub/home)へ遷移して、`3-1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。<br>
実行環境の停止・削除後は、当実行環境にあるノートブックを操作することが出来なくなりますのでご注意ください。<br>
※ `delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。<br>
![コンテナ削除キャプチャ](./images/コンテナ削除キャプチャ.png)

### 3-3. GIN-forkに実行環境の停止・削除を反映する
以下を実行して、GIN-forkに実行環境の停止・削除を反映してください。

In [ ]:
import json, requests
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

file_path = os.environ['HOME'] + '/.repository_id'

with open('/home/jovyan/.token.json', 'r') as f:
        dic = json.load(f)
        token = dic["ginfork_token"]

server_name = os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3]
response = requests.delete(
    params['siblings']['ginHttp'] + f'/api/v1/container?token={token}&server_name={server_name}'
)

if response.status_code == requests.codes.ok:
    display_util.display_info('実行環境の削除を反映しました。')
else:
    display_util.display_err('反映に失敗しました。')